# オペレーティングシステム 演習 6
# ページ置換アルゴリズムの観察

**このセルを編集して, 名前と学生証番号を書け.**

 * 名前 Name : 03-190503
 * 学生証番号 Student ID : 西山　晃人

**書けたら Shift + Enter で実行(入力を確定)させ, 保存(`Ctrl-S`)せよ**

## 1. はじめに

* 演習 5 において, 物理メモリ量 (通常は搭載物理メモリ量. ただし先の実験においてはcgroups を用いて 256MB という人工的な制限を課していた) を上回るメモリ領域をアクセスするとメジャーページフォルトが多数発生し, 急激に性能が低下することを見た

* OSがどのページをメモリ上に置くかのアルゴリズムの基本は, 「古いものを捨て, 新しいものは残す」ということ
 * 古い = 長らくアクセスされていない, 新しい = 最近アクセスされた, ということ
* これを完全に行ったのが LRU (Least Recently Used)置換というもの
 * LRU置換では, 捨てなくてはいけないページを選ぶ際, 最後にアクセスされたのがもっとも古い時点であるものを捨てる
* 実際のOSはLRU置換を近似している
 * 厳密な順番を管理する代わりに, ある頻度で区間を区切り, 各区間でのアクセスの有無を記録する

## 2. 常駐メモリの取得と記録

* Linuxにはmincoreというシステムコールがあり, 指定した範囲のどのページに, 現在物理メモリが割り当てられているかを知ることが出来る
* 以下はそれを利用して,
 * A MBの配列を用意
 * その全ページを順にアクセスする. それをB回繰り返す
 * 時折, mincore を読んで配列中のどの部分が物理メモリにあるかを記録する

ということをするプログラム

In [1]:
//% file: record_incore_1.c
//% cmd: gcc -O3 -Wall -Wextra -o record_incore_1 record_incore_1.c


#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <unistd.h>
#include <sys/mman.h>

void die(const char * s) {
  perror(s);
  exit(1);
}

void die2(int err, const char * s) {
  fprintf(stderr, "%s: %s\n", s, strerror(err));
  exit(1);
}

/* ページサイズ */
const long page_sz = 1 << 12;     /* 4096 */

/* 秒単位で時刻を返す.
   初めて呼び出された時が時刻0 */
double cur_time() {
  static long t0 = 0;
  struct timeval tp[1];
  gettimeofday(tp, 0);
  long t = tp->tv_sec * 1000000 + tp->tv_usec;
  if (t0 == 0) t0 = t;
  return (t - t0) * 1.0e-6;
}

typedef struct {
  FILE * wp;                    /* 保存するファイル */
  void * a;                     /* アクセスしている配列の先頭ページ */
  long sz;                      /* アクセスしている配列のサイズ(ページの先頭から最後まで) */
  unsigned char * v;            /* mincoreの結果を受け取るバッファ */
  long n_pages;                 /* ページ数(vの要素数) */
} incore_records_t;

/* a から始まる sz バイトに対応する記録を準備 */
incore_records_t * mk_incore_records(void * a, long sz, const char * filename) {
  long begin = (long)a;
  long end   = (long)(a + sz + page_sz - 1);
  void * begin_aligned = (void*) (begin - begin % page_sz); /* aを含むページの先頭 */
  void * end_aligned   = (void*) (end   - end % page_sz);   /* a+sz-1を含むページの次のページの先頭 */
  long sz_aligned = end_aligned - begin_aligned;
  long n_pages = sz_aligned / page_sz; /* ページ数 */
  unsigned char * v = calloc(1, n_pages); /* mincoreの結果を受け取るバッファ */
  if (!v) die("malloc");
  FILE * wp = fopen(filename, "w"); /* 結果を書き込む */
  if (!wp) die("fopen");
  incore_records_t * ir = malloc(sizeof(incore_records_t));
  if (!ir) die("malloc");
  ir->a = begin_aligned;
  ir->sz = sz_aligned;
  ir->n_pages = n_pages;
  ir->v = v;
  ir->wp = wp;
  return ir;
}

/* 時刻やページフォルトの回数を記録 */
void record_incore(incore_records_t * ir, void * last_addr) {
  /* get current time */
  double t = cur_time();
  unsigned char * v = ir->v;
  long n_pages = ir->n_pages;
  FILE * wp = ir->wp;
  if (mincore(ir->a, ir->sz, v) == -1) die("mincore");

  /* 記録の仕方: 0がA個, 1がB個, 0がC個, 1がD個, ... に対して A B C D ...
     を記録する(run length圧縮).
     それ以外に時刻と, 直近にアクセスした場所(何ページ目か)を記録 */
  unsigned char c = 0;
  size_t x = 0;
  fprintf(wp, "%f", t);         /* 時刻 */
  fprintf(wp, " %lu", (last_addr - ir->a) / page_sz); /* 直近のページ */
  /* run-length 符号化 */
  for (long i = 0; i < n_pages; i++) {
    if (v[i] != c) {
      fprintf(wp, " %lu", x);
      c = v[i];
      x = 0;
    }
    x++;
  }
  assert(x > 0);
  fprintf(wp, " %lu", x);
  /* 改行 */
  fprintf(wp, "\n");
}

/* 記録された時刻とページフォルト数をファイルに保存 */
void save_records(incore_records_t * ir) {
  fclose(ir->wp);
}

/* 記録の間隔(アクセス数). 1MB分のページを触る毎に記録 */
const long record_interval = 59;

/* A MB 割り当ててそれを B 回スキャンする.
   記録を log に書く */
int main(int argc, char ** argv) {
  long A = (argc > 1 ? atol(argv[1]) : 128);
  long B = (argc > 2 ? atol(argv[2]) : 3);
  char * log = (argc > 3 ? argv[3] : "incore.log");
  long sz = A * 1024L * 1024L;
  cur_time();
  /* A MB割当て. posix_memalign で先頭アドレスがページの先頭に来るように */
  char * a = 0;
  int err = posix_memalign((void **)&a, page_sz, sz);
  if (err) die2(err, "posix_memalign");
  incore_records_t * ir = mk_incore_records(a, sz, log);
  /* (A MB 中のページ数) x B 回 ページをアクセス */
  long n_accesses = sz / page_sz * B;
  long j = 0;
  for (long k = 0; k < n_accesses; k++) {
    a[j % sz]++;
    if (k % record_interval == 0) record_incore(ir, &a[j % sz]);
    j += page_sz;
  }
  save_records(ir);
  return 0;
}


199MB を 3回

In [2]:
//% cmd: /usr/bin/time ./record_incore_1 199 3 incore199x3.log 2>&1

0.11user 1.15system 0:12.93elapsed 9%CPU (0avgtext+0avgdata 205452maxresident)k
16448inputs+488outputs (2046major+54815minor)pagefaults 0swaps


実行したら, os06_page_replacement_visualize.ipynb を開いて, 可視化してみよ

293MB を 3回

In [3]:
//% cmd: /usr/bin/time ./record_incore_1 293 3 incore293x3.log 2>&1

0.77user 75.66system 2:40.94elapsed 47%CPU (0avgtext+0avgdata 231828maxresident)k
1200256inputs+26552outputs (149996major+75120minor)pagefaults 0swaps


同様に, os06_page_replacement_visualize.ipynb を開いて, 可視化してみよ

## <font color="blue">課題 6-1:</font> アクセスパターンに変化を持たせてみる

* 全ページを頭から終わりまでアクセスするという単純過ぎるアクセスパターンに少し変化を持たせてみる

* 以下を変更して, ページを5つ飛ばしでアクセスするようにせよ

* 注: プログラムの挙動について
```
./record_incore_2 A B FILE
```
とすると, 
 * A MBの配列を用意
 * (A MB / ページサイズ * B) 回, 配列の要素をアクセスする
 * j 回目のアクセスは,  a[j * 5 * ページサイズ % 配列サイズ] をアクセスする(したがって1週目と2周目では違う要素をアクセスするかもしれない)

* 可視化したらどんな結果が見えるかを予想してみよ

In [5]:
//% file: record_incore_2.c
//% cmd: gcc -O3 -Wall -Wextra -o record_incore_2 record_incore_2.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <unistd.h>
#include <sys/mman.h>

void die(const char * s) {
  perror(s);
  exit(1);
}

void die2(int err, const char * s) {
  fprintf(stderr, "%s: %s\n", s, strerror(err));
  exit(1);
}

/* ページサイズ */
const long page_sz = 1 << 12;     /* 4096 */

/* 秒単位で時刻を返す.
   初めて呼び出された時が時刻0 */
double cur_time() {
  static long t0 = 0;
  struct timeval tp[1];
  gettimeofday(tp, 0);
  long t = tp->tv_sec * 1000000 + tp->tv_usec;
  if (t0 == 0) t0 = t;
  return (t - t0) * 1.0e-6;
}

typedef struct {
  FILE * wp;                    /* 保存するファイル */
  void * a;                     /* アクセスしている配列の先頭ページ */
  long sz;                      /* アクセスしている配列のサイズ(ページの先頭から最後まで) */
  unsigned char * v;            /* mincoreの結果を受け取るバッファ */
  long n_pages;                 /* ページ数(vの要素数) */
} incore_records_t;

/* a から始まる sz バイトに対応する記録を準備 */
incore_records_t * mk_incore_records(void * a, long sz, const char * filename) {
  long begin = (long)a;
  long end   = (long)(a + sz + page_sz - 1);
  void * begin_aligned = (void*) (begin - begin % page_sz); /* aを含むページの先頭 */
  void * end_aligned   = (void*) (end   - end % page_sz);   /* a+sz-1を含むページの次のページの先頭 */
  long sz_aligned = end_aligned - begin_aligned;
  long n_pages = sz_aligned / page_sz; /* ページ数 */
  unsigned char * v = calloc(1, n_pages); /* mincoreの結果を受け取るバッファ */
  if (!v) die("malloc");
  FILE * wp = fopen(filename, "w"); /* 結果を書き込む */
  if (!wp) die("fopen");
  incore_records_t * ir = malloc(sizeof(incore_records_t));
  if (!ir) die("malloc");
  ir->a = begin_aligned;
  ir->sz = sz_aligned;
  ir->n_pages = n_pages;
  ir->v = v;
  ir->wp = wp;
  return ir;
}

/* 時刻やページフォルトの回数を記録 */
void record_incore(incore_records_t * ir, void * last_addr) {
  /* get current time */
  double t = cur_time();
  unsigned char * v = ir->v;
  long n_pages = ir->n_pages;
  FILE * wp = ir->wp;
  if (mincore(ir->a, ir->sz, v) == -1) die("mincore");

  /* 記録の仕方: 0がA個, 1がB個, 0がC個, 1がD個, ... に対して A B C D ...
     を記録する(run length圧縮).
     それ以外に時刻と, 直近にアクセスした場所(何ページ目か)を記録 */
  unsigned char c = 0;
  size_t x = 0;
  fprintf(wp, "%f", t);         /* 時刻 */
  fprintf(wp, " %lu", (last_addr - ir->a) / page_sz); /* 直近のページ */
  /* run-length 符号化 */
  for (long i = 0; i < n_pages; i++) {
    if (v[i] != c) {
      fprintf(wp, " %lu", x);
      c = v[i];
      x = 0;
    }
    x++;
  }
  assert(x > 0);
  fprintf(wp, " %lu", x);
  /* 改行 */
  fprintf(wp, "\n");
}

/* 記録された時刻とページフォルト数をファイルに保存 */
void save_records(incore_records_t * ir) {
  fclose(ir->wp);
}

/* 記録の間隔(アクセス数). 1MB分のページを触る毎に記録 */
const long record_interval = 59;

/* A MB 割り当ててそれを B 回スキャンする.
   記録を log に書く */
int main(int argc, char ** argv) {
  long A = (argc > 1 ? atol(argv[1]) : 128);
  long B = (argc > 2 ? atol(argv[2]) : 3);
  char * log = (argc > 3 ? argv[3] : "incore.log");
  long sz = A * 1024L * 1024L;
  cur_time();
  /* A MB割当て. posix_memalign で先頭アドレスがページの先頭に来るように */
  char * a = 0;
  int err = posix_memalign((void **)&a, page_sz, sz);
  if (err) die2(err, "posix_memalign");
  incore_records_t * ir = mk_incore_records(a, sz, log);
  /* (A MB 中のページ数) x B 回 ページをアクセス */
  long n_accesses = sz / page_sz * B;
  long j = 0;
  for (long k = 0; k < n_accesses; k++) {
    a[j % sz]++;
    if (k % record_interval == 0) record_incore(ir, &a[j % sz]);
    /* 以下を書き換えて 5 ページ飛ばしでアクセスするようにせよ */
    j += page_sz * 5;
  }
  save_records(ir);
  return 0;
}

//予想：ページアウトされている領域が先の例に比べてさらにまばらになる。


199MB を 3回

In [6]:
//% cmd: /usr/bin/time ./record_incore_2 199 3 incore199_5x3.log 2>&1

1.37user 1.07system 0:23.13elapsed 10%CPU (0avgtext+0avgdata 205392maxresident)k
5496inputs+56104outputs (687major+53978minor)pagefaults 0swaps


実行したら, os06_page_replacement_visualize.ipynb を開いて, 可視化してみよ

293MB を 3回

In [7]:
//% cmd: /usr/bin/time ./record_incore_2 293 3 incore293_5x3.log 2>&1

9.31user 73.73system 2:59.56elapsed 46%CPU (0avgtext+0avgdata 247084maxresident)k
1209552inputs+503696outputs (136307major+88813minor)pagefaults 0swaps


295MB を 3回 (293 MB との違いに注目. なぜか?):理由...295が5の倍数になっており、毎週同じ場所をアクセスしているため物理メモリは295/5MB程度しか使わず、既存のメモリ領域のみで処理することができメジャーフォルトが起こらない。

In [8]:
//% cmd: /usr/bin/time ./record_incore_2 295 3 incore295_5x3.log 2>&1

8.24user 1.27system 0:09.52elapsed 99%CPU (0avgtext+0avgdata 62096maxresident)k
0inputs+438088outputs (0major+15186minor)pagefaults 0swaps
